# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [10]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os
import re

In [11]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {
    "journal":{
        "file": "pubs.bib",
        "venuekey" : "journal",
        "venue-pretext" : "",
        "collection" : {"name":"publications",
                        "permalink":"/publication/"}
    },
#     "proceeding": {
#         "file" : "proceedings.bib",
#         "venuekey": "booktitle",
#         "venue-pretext": "In the proceedings of ",
#         "collection" : {"name":"publications",
#                         "permalink":"/publication/"}
        
#     }
}

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [13]:
publist['journal']["file"]

'pubs.bib'

In [14]:
parser = bibtex.Parser()
bibdata = parser.parse_file(publist[pubsource]["file"])

#loop through the individual references in a given bibtex file
for bib_id in bibdata.entries:
    #reset default date
    pub_year = "1900"
    pub_month = "01"
    pub_day = "01"

    b = bibdata.entries[bib_id].fields

NameError: name 'pubsource' is not defined

In [15]:
b

NameError: name 'b' is not defined

In [17]:
for pubsource in publist:
    parser = bibtex.Parser()
    bibdata = parser.parse_file(publist[pubsource]["file"])

    #loop through the individual references in a given bibtex file
    for bib_id in bibdata.entries:
        #reset default date
        pub_year = "1900"
        pub_month = "01"
        pub_day = "01"
        
        b = bibdata.entries[bib_id].fields
        
        try:
            pub_year = f'{b["year"]}'

            #todo: this hack for month and day needs some cleanup
            if "month" in b.keys(): 
                if(len(b["month"])<3):
                    pub_month = "0"+b["month"]
                    pub_month = pub_month[-2:]
                elif(b["month"] not in range(12)):
                    tmnth = strptime(b["month"][:3],'%b').tm_mon   
                    pub_month = "{:02d}".format(tmnth) 
                else:
                    pub_month = str(b["month"])
            if "day" in b.keys(): 
                pub_day = str(b["day"])

                
            pub_date = pub_year+"-"+pub_month+"-"+pub_day
            
            #strip out {} as needed (some bibtex entries that maintain formatting)
            clean_title = b["title"].replace("{", "").replace("}","").replace("\\","").replace(" ","-")    

            
            url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
            url_slug = url_slug.replace("--","-")

            md_filename = (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
            html_filename = (str(pub_date) + "-" + url_slug).replace("--","-")

            
            
            #########################################
            # citation authors
            author_citation = ""
            authors = bibdata.entries[bib_id].persons["author"]
            for author in authors:
                author_citation += " "\
                            +author.first_names[0].replace("{", "").replace("}","").replace("\\","")+" "\
                            +author.last_names[0].replace("{", "").replace("}","").replace("\\","")+", "
                
            if len(authors) > 10:
                author_citation = authors[0].first_names[0].replace("{", "").replace("}","").replace("\\","")+" "\
                            +authors[0].last_names[0].replace("{", "").replace("}","").replace("\\","")+" and others, "

            ##########################################
            # Build Citation from text
            citation = ""

            # citation title
            citation += "\"" + b["title"].replace("{", "").replace("}","").replace("\\","").replace("\"","") + "\","
            
            # citation author list
            citation += author_citation 
            
            # add venue logic depending on citation type
            venue = publist[pubsource]["venue-pretext"]+b[publist[pubsource]["venuekey"]].replace("{", "").replace("}","").replace("\\","")
            
            if venue == 'prd':
                venue = '<strong>PRD</strong>'
            if venue == 'prl':
                venue = '<strong>PRL</strong>'
            if venue == 'arXiv e-prints':
                venue = '<em>arXiv preprint</em>'

            citation += " " + html_escape(venue)
            citation += ", " + pub_year

            ##########################################
            ## YAML variables
            md = "---\ntitle: \""   + b["title"].replace("{", "").replace("}","").replace("\\","")+ '"\n'

            md += """authors: """   + html_escape(author_citation)[:-2] + '\n'

            md += """collection: """ +  publist[pubsource]["collection"]["name"]

            md += """\npermalink: """ + publist[pubsource]["collection"]["permalink"]  + html_filename
            
            note = False
            if "note" in b.keys():
                if len(str(b["note"])) > 5:
                    md += "\nexcerpt: '" + html_escape(b["note"]) + "'"
                    note = True

            md += "\ndate: " + str(pub_date) 

            md += "\nvenue: '" + html_escape(venue) + "'"
            
            url = False
            if "url" in b.keys():
                if len(str(b["url"])) > 5:
                    md += "\npaperurl: '" + b["url"] + "'"
                    url = True
            
            md += "\ncitation: '" + citation + ", '"

            md += "\neprint: '" + html_escape(b["eprint"]) +"'\n"

            md += "---"
                
            md_filename = os.path.basename(md_filename)

            with open("../_publications/" + md_filename, 'w') as f:
                f.write(md)
            print(f'SUCESSFULLY PARSED {bib_id}: \"', b["title"][:60],"..."*(len(b['title'])>60),"\"")
        # field may not exist for a reference
        except KeyError as e:
            print(f'WARNING Missing Expected Field {e} from entry {bib_id}: \"', b["title"][:30],"..."*(len(b['title'])>30),"\"")
            continue


SUCESSFULLY PARSED 2020RPPh...83l4201A: " {New opportunities at the next-generation neutrino experimen ... "
SUCESSFULLY PARSED 2020arXiv201202142H: " {Novel multi-lepton signatures of dark sectors in light meso ... "
SUCESSFULLY PARSED 2020PhRvD.102e5016H: " {Pair production of dark particles in meson decays}  "
SUCESSFULLY PARSED 2020arXiv200811851H: " {Constraints on Decaying Sterile Neutrinos from Solar Antine ... "
SUCESSFULLY PARSED Abdullahi:2020nyr: " {A dark seesaw solution to low energy anomalies: MiniBooNE,  ... "
SUCESSFULLY PARSED 2020PhRvD.101k5025B: " {Dark neutrinos and a three-portal connection to the standar ... "
SUCESSFULLY PARSED 2019PhRvL.123z1801A: " {Testing New Physics Explanations of the MiniBooNE Anomaly a ... "
SUCESSFULLY PARSED 2019PhRvD.100e5012B: " {Z$^{'}$s in neutrino scattering at DUNE}  "
SUCESSFULLY PARSED 2019arXiv190700991B: " {Neutrino Non-Standard Interactions: A Status Report}  "
SUCESSFULLY PARSED 2019PhRvD..99i1701B: " {Neutrino masses from a